<a href="https://colab.research.google.com/github/flaaa31/meetings_handling/blob/main/meetings_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Riassunto e Identificazione delle Attività dalla Trascrizione di un Meeting
## Introduzione
La capacità di analizzare le trascrizioni dei meeting aziendali è fondamentale per organizzare il lavoro e identificare le azioni chiave. In questo progetto, lo studente implementerà un sistema basato su modelli di linguaggio di grandi dimensioni (LLM) come GPT o LLama per processare una trascrizione e generare un riassunto e un elenco strutturato delle attività.

## Obiettivo del Progetto
Realizzare uno script in Python che:
1. Analizzi una trascrizione di un meeting fornita (disponibile al link: https://raw.githubusercontent.com/Profession-AI/progetti-llm/refs/heads/main/Riassunto%20e%20identificazione%20delle%20attivit%C3%A0%20dalla%20trascrizione%20di%20un%20meeting/meeting_transcription.txt).
2. Generi un riassunto conciso dei punti principali discussi.
3. Identifichi e strutturi le attività assegnate a ciascun partecipante.

## Requisiti
* Input: Un file di testo contenente la trascrizione del meeting (es. meeting_transcription.txt).
* Output:
  * Un riassunto dei punti chiave del meeting.
  * Un elenco di attività assegnate, con indicazione della persona responsabile.

## Linee Guida Tecniche
1. Scelta del Modello:

  * Utilizzare un modello di linguaggio LLM come GPT o LLama. Si consiglia l'uso di API di OpenAI o un'implementazione open-source di LLaMA.
2. Pipeline del Progetto:

  * Pre-processing: Caricare il file di trascrizione.
  * Riassunto: Utilizzare il modello LLM per generare un riassunto dei contenuti principali.
  * Estrazione delle Attività: Analizzare il testo per identificare frasi che indicano compiti assegnati, azioni future o responsabilità.
3. Struttura dell'Output:

  * Riassunto:
  Riassunto del Meeting:
    - Punto 1...
    - Punto 2...
  * Elenco delle Attività:
  Attività Identificate:
    - Mario Rossi: Redigere un piano tecnico basato sui requisiti emersi.
    - Andrea Monti: Implementare un modulo per le notifiche e il server SMTP.
4. Testing e Validazione:

  * Verificare che il sistema sia in grado di identificare correttamente attività e responsabilità, con un livello di accuratezza accettabile.

## Consegna
* Uno script Python documentato.
* Il riassunto generato e l'elenco delle attività basati sul file di trascrizione fornito.
* Un breve report che descriva:
  * Il funzionamento dello script.
  * I risultati ottenuti.
  * Le eventuali limitazioni del sistema.

# Initial setup

In [ ]:
!pip install langchain llama_index dotenv --q
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.6/263.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 3.2 MB/s eta 0:00:00


# Loading GROQ_API_KEY, used for LLama3 model, the only LLM used in this project
it is free (with usage limitations) api keys can be created [here](https://console.groq.com/keys)


In [ ]:
# Loading GROQ_API_KEY, used for LLama3 model (only LLM used in this project), using a .env file
from dotenv import load_dotenv
import os
load_dotenv("/content/drive/MyDrive/ProfessionAI/Master AI Developer/7.LLM/progetto/.env") # better to create a .env file, instead of showing api keys in the code, for safety reasons

GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

In [ ]:
# meeting file download
! wget "https://raw.githubusercontent.com/Profession-AI/progetti-llm/refs/heads/main/Riassunto%20e%20identificazione%20delle%20attivit%C3%A0%20dalla%20trascrizione%20di%20un%20meeting/meeting_transcription.txt"

--2025-05-15 07:44:12--  https://raw.githubusercontent.com/Profession-AI/progetti-llm/refs/heads/main/Riassunto%20e%20identificazione%20delle%20attivit%C3%A0%20dalla%20trascrizione%20di%20un%20meeting/meeting_transcription.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2518 (2.5K) [text/plain]
Saving to: ‘meeting_transcription.txt’

meeting_transcripti 100%[===================>]   2.46K  --.-KB/s    in 0s      

2025-05-15 07:44:12 (39.1 MB/s) - ‘meeting_transcription.txt’ saved [2518/2518]



In [ ]:
# every file in the current folder will be included in "documents" list, using SimpleDirectoryReader
from llama_index.core import SimpleDirectoryReader
folder= '.' # current directory, where we download "meeting_transcription.txt"

# Loading documents
documents = SimpleDirectoryReader(folder).load_data()

In [ ]:
# only one document in our folder, no need to worry about choosing the wrong file
len(documents)

1

In [ ]:
# Showing first and only document
print(documents[0].text)

Mario Rossi: Buongiorno a tutti. Come sapete, siamo qui per discutere i requisiti del nuovo sistema contabile. Rossana, potresti iniziare spiegandoci le principali necessità dal punto di vista dell’utente?

Rossana Bolletta: Certo, Mario. Attualmente il nostro sistema è molto macchinoso. Abbiamo bisogno di un'interfaccia più intuitiva per l’inserimento delle fatture, una gestione automatizzata delle scadenze dei pagamenti e una reportistica personalizzabile.

Mario Rossi: Capisco. Quando parli di reportistica personalizzabile, intendi la possibilità di scegliere i parametri da analizzare, come intervalli di tempo o categorie di spesa?

Rossana Bolletta: Esatto. Al momento siamo costretti a estrarre i dati e modificarli manualmente in Excel. Sarebbe ideale poter generare i report direttamente dal sistema, filtrando secondo le nostre esigenze.

Andrea Monti: Questo implica la necessità di un modulo dedicato ai filtri e ai grafici. Inoltre, per l’automazione delle scadenze, immagino serva

# Generating meeting summary and tasks using Langchain

In [ ]:
meeting_text = documents[0].text

In [ ]:
# model definition, in our case llama3-70b-8192, but we can use other models if we want
from langchain_groq import ChatGroq

llama3 = ChatGroq(
    api_key = GROQ_API_KEY,
    model_name = "llama3-70b-8192")

"""
# we could also use openai models, for example gpt-4o-mini but we need a paid account and generating the api_key from their site (https://platform.openai.com/api-keys)
gpt4o = ChatOpenAI(
    api_key = os.environ.get("OPENAI_API_KEY"),
    model_name = "gpt-4o-mini")
"""

'\n# we could also use openai models, for example gpt-4o-mini but we need a paid account and generating the api_key from their site (https://platform.openai.com/api-keys)\ngpt4o = ChatOpenAI(\n    api_key = os.environ.get("OPENAI_API_KEY"),\n    model_name = "gpt-4o-mini")\n'

In [ ]:
# Zero shot example, no meeting file given, expecting it will say the deafult answer
messages = [
    (
      "system",
      "Sei un chatbot specializzato nell'aiutare gli utenti con delle informazioni sui meeting, se non viene data nessuna informazione sul meeting, rispondi solamente con: 'Scusa, non ho ancora ricevuto alcuna discussione, quindi non posso riassumere nulla'",
    ),
    (
     "human",
     "Parlami brevemente del meeting che è stato fatto."
     )]

llama_zeroshot = llama3.invoke(messages)

print(llama_zeroshot.content)

Scusa, non ho ancora ricevuto alcuna discussione, quindi non posso riassumere nulla.


without any file given, the model correctly says that there's no file, and it doesn't make any allucinations

## Let's see if the model "sees" the meeting file

In [ ]:
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq


# Generic prompt template
PROMPT_MEETING = """\
Sei un chatbot specializzato nell'aiutare gli utenti con delle informazioni sui meeting. \
Se non viene data nessuna informazione sul meeting, rispondi solamente con: \
'Scusa, non ho ancora ricevuto alcuna discussione, quindi non posso riassumere nulla'.

Ecco il testo del meeting:
'''{meeting_text}'''
"""

prompt_meeting = PromptTemplate(
    input_variables=["meeting_text"],
    template=PROMPT_MEETING
)

# Building the runnable chain: prompt | model | output parsing as string
meeting_chain = prompt_meeting | llama3 | StrOutputParser()

# Chain invocation, passing meeting_text
result = meeting_chain.invoke({"meeting_text": meeting_text})

print(result)  # model response


Ecco il riassunto del meeting:

Il meeting è stato convocato per discutere i requisiti del nuovo sistema contabile. I principali requisiti emersi sono:

* Un'interfaccia più intuitiva per l’inserimento delle fatture
* Una gestione automatizzata delle scadenze dei pagamenti
* Una reportistica personalizzabile
* Un modulo dedicato ai filtri e ai grafici
* Notifiche e promemoria per le scadenze delle fatture
* Invio email automatiche ai fornitori in caso di ritardi nei pagamenti
* Un'archiviazione digitale dei documenti con possibilità di allegare PDF delle fatture
* Una gestione documentale con funzione di ricerca avanzata

Andrea Monti si occuperà di redigere un piano tecnico basato sui requisiti emersi e ci sarà un altro incontro per approvare il progetto definitivo.


the model talks about the correct discussion file, without any allucinations

# Summary generation and task definition
- task1: meeting summary
- task2: tasks extraction for every person

In [ ]:
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq


# Task 1: meeting summary
PROMPT_SUMMARY = """\
Sei un chatbot specializzato nel riassumere i meeting.
Genera un riassunto conciso dei punti principali discussi nel meeting:

L'output deve avere il seguente formato:
Riassunto del Meeting:
- Punto 1...
- Punto 2...

Non scrivere nulla prima o dopo l'output.
Ecco il testo del meeting:
'''{meeting_text}'''
"""

prompt_summary = PromptTemplate(
    input_variables=["meeting_text"],
    template=PROMPT_SUMMARY
)

# Task 2: tasks extraction
PROMPT_TASKS = """\
Partendo da questo meeting: {meeting_text}, identifica e struttura tutte le attività assegnate ai partecipanti.
L'output deve avere il seguente formato:
Attività Identificate:
- Partecipante 1: Attività (descrizione sintetica)...
- Partecipante 2: Attività (descrizione sintetica)...

Se il partecipante non ha task assegnati, metti "Nessun task" come descrizione.
Non scrivere nulla prima o dopo l'output.
"""

prompt_tasks = PromptTemplate(
    input_variables=["meeting_text"],
    template=PROMPT_TASKS
)

# Building two chains, one for summary and one for tasks
chain_summary = prompt_summary | llama3 | StrOutputParser()
chain_tasks = prompt_tasks | llama3 | StrOutputParser()


# Invoking of two chains
result_1 = chain_summary.invoke({"meeting_text": meeting_text})
result_2 = chain_tasks.invoke({"meeting_text": meeting_text})

# Printing both Results
print("Riassunto:\n", result_1)
print("\nAttività:\n", result_2)

# Saving results as a markdown_file
final_result = result_1 + "\n\n" + result_2
with open("final_result.md", "w") as f:
    f.write(final_result)

# display saving message
print("\nResults saved as 'final_result.md'")


Riassunto:
 Riassunto del Meeting:
- Il nuovo sistema contabile deve avere un'interfaccia più intuitiva per l’inserimento delle fatture.
- È necessaria una gestione automatizzata delle scadenze dei pagamenti con notifiche e promemoria.
- La reportistica deve essere personalizzabile con la possibilità di scegliere i parametri da analizzare.
- È richiesta una funzione di archiviazione digitale dei documenti con allegazione di PDF delle fatture.
- Sarà necessario uno spazio di storage e una gestione documentale con funzione di ricerca avanzata.
- Andrea Monti sarà responsabile della redazione di un piano basato sui requisiti emersi.

Attività:
 Attività Identificate:
- Rossana Bolletta: Nessun task
- Andrea Monti: Integrare sistema di notifiche e invio email, implementare funzione di ricerca avanzata per documenti, redigere piano tecnico
- Mario Rossi: Nessun task

Results saved as 'final_result.md'


# Final solution, in a more professional way
main function that:
- download discussion file from URL, or load it from a local file
- query to LLM model, handling big files and too many requests in a short amount of time (chunk division and retry with exponential time)
- summary of the discussion meeting
- tasks extraction for every person in the meeting
- translates everything in a language of choice (italian, in this case)
- display and saves results in a markdown file

For this case, I generated using ChatGPT a long meeting file ("long_meeting.txt"), it doesn't make that much sense, it starts talking about a data scientist meeting and then it goes on and on with other stuff to test the chunking of text that is too large for the LLM context window, so the content will not make that much sense, but it's only for educational purposes

It can work also with the previous meeting file, if we set the "is_url" parameter to True

In [ ]:
# Libraries import
import os
import requests
import json
import textwrap
import time
import random
from IPython.display import Markdown, display


# API configuration
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

# API URL
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"


def query_llama(prompt, max_tokens=4000, temperature=0.1, max_retries=5, initial_delay=2):
    """
    Function that calls the LLama3-70b model using Groq with exponential backoff and retry.

    Parameters:

    prompt(str): Prompt to give
    max_tokens(int): Maximum number of tokens to generate, default is 4000
    temperature(float): Temperature for sampling, default is 0.1
    max_retries(int): Maximum number of retries, default is 5
    initial_delay(int): Initial delay in seconds before the first retry, default is 2

    Returns:
    str: Generated text
    """
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}", # api_key for authentication
        "Content-Type": "application/json" # to specify that we're gonna pass a json file as a body
    }

    data = {
        "model": "llama3-70b-8192", # the same model as before
        "messages": [{"role": "user", "content": prompt}], # chat structure
        "max_tokens": max_tokens, # default max tokens (4000)
        "temperature": temperature # default temperature (0.1), as low as we can in order to not obtain too creative responses
    }

    retry_count = 0 # how many times the request has been repeated
    delay = initial_delay # waiting time before retry, starts from 2 by default

    while retry_count < max_retries:
        try:
            # Adding a delay before every API call to avoid rate limiting
            if retry_count > 0:
                print(f"Waiting {delay} seconds before attempt {retry_count + 1}...") # every retry, prints the delayed seconds
            time.sleep(delay) # and wait that many seconds

            response = requests.post(GROQ_API_URL, headers=headers, data=json.dumps(data)) # sending request to GROQ endpoint

            # If we receive Status Code 429 (Too Many Requests), we wait and retry
            if response.status_code == 429:
                retry_count += 1
                # Exponential backoff plus a random jitter to avoid synchronized peaks
                delay = min(60, delay * 2) + random.uniform(0, 1)
                print(f"Rate limiting (429). Retry in {delay:.2f} seconds... (attempt {retry_count}/{max_retries})")
                continue

            response.raise_for_status()  # Raise Exception for other HTTP Errors
            return response.json()["choices"][0]["message"]["content"] # returns generated JSON

        except requests.exceptions.HTTPError as e: # HTTP exceptions handling
            # printing error and body
            print(f"HTTP Error during API call: {e}")
            if 'response' in locals() and response:
                print(f"Server response: {response.text}")

            # If It's not 429 Error, or we have exhausted attempts, break cycle
            if not (response.status_code == 429 and retry_count < max_retries):
                break

        except Exception as e: # Generic exceptions, same handling as before
            print(f"Error during API call: {e}")
            if 'response' in locals() and response:
                print(f"Server response: {response.text}")
            break

        retry_count += 1 # incrementing retry count

    print(f"Impossible to complete request after {max_retries} attempts.") # "too many attempts" error message
    return None

def download_discussion(url):
    """
    Download discussion file from URL.

    Parameters:
    url(str): URL of the discussion file.

    Returns:
    str: Text content of the discussion file.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except Exception as e:
        print(f"Error during discussion download: {e}")
        return None


def split_text_into_chunks(text, max_chunk_size=4000):
    """
    Split text into chunks of a given maximum size.

    Parameters:
    text(str): Text to split.
    max_chunk_size(int): Maximum size of each chunk.

    Returns:
    list: List of chunks.
    """
    # Dividing for each line
    lines = text.split("\n")
    chunks = [] # list to collect all the chunks
    current_chunk = "" # temporary string to collect strings until max dimension is reached

    for line in lines:
        # If adding this line surpasses maximum length, we save the actual chunk
        if len(current_chunk) + len(line) + 1 > max_chunk_size and current_chunk:
            chunks.append(current_chunk) # adding current_chunk to chunks list
            current_chunk = line  # starting new chunk with the line that surpassed the maximum length
        else:
            if current_chunk:
                current_chunk += "\n" + line # if it doesn't surpass maximum length, we add the line to the current chunks list
            else:
                current_chunk = line # if current chunk is empty, we start with this line, without adding new lines

    # If we have a last not empty current_chunk, we append it to chunks list
    if current_chunk:
        chunks.append(current_chunk)

    print(f"Discussion divided in {len(chunks)} chunks.")
    return chunks

def generate_summary_from_chunks(chunks, max_chunk_size=6000):
    """
    Generate a summary from a list of chunks.

    Parameters:
    chunks(list): List of text chunks.
    max_chunk_size(int): Maximum size of each chunk.

    Returns:
    str: Generated summary.
    """
    # First, we obtain partial results for every chunk
    partial_summaries = [] # list of indivisual summaries of every chunk

    for i, chunk in enumerate(chunks):
        print(f"Elaboration of chunk {i+1}/{len(chunks)}...")
        prompt = f"""\
                  Sei un chatbot specializzato nel riassumere i meeting.
                  Genera un riassunto conciso dei punti principali discussi in questa PARTE del meeting:
                  '''{chunk}'''

                  Tieni presente che questo è solo un frammento dell'intera discussione.
                  Riassumi i punti chiave trattati in questa parte.
                  """

        partial_summary = query_llama(prompt, temperature=0.2) # model calling
        if partial_summary:
            partial_summaries.append(partial_summary) # if we generate a summary, we append it to partial_summary
            # Pause between chunk elaborations to avoid rate limiting
            if i < len(chunks) - 1:  # No Pause after last chunk
                pause_time = 2 + random.uniform(0, 1)  # Default Pause + jitter
                print(f"Waiting {pause_time:.2f} seconds before next chunk...")
                time.sleep(pause_time)

    # Final summary obtained joining all partial summaries
    if partial_summaries:
        combined_summaries = "\n\n".join(partial_summaries)

        # If combined_summaries is still too long, we have to divide it again
        if len(combined_summaries) > max_chunk_size:
            print("Combined Partial Summaries still too long, dividing again...")
            summary_chunks = split_text_into_chunks(combined_summaries, max_chunk_size) # break partial summaries in more chunks
            final_summaries = []

            # union of partial mini-summaries in final_summaries
            for i, summary_chunk in enumerate(summary_chunks):
                prompt = f"""\
                          Combina questi riassunti parziali in un unico riassunto coerente:
                          '''{summary_chunk}'''

                          L'output deve avere il seguente formato:
                          Riassunto del Meeting:
                          - Punto 1...
                          - Punto 2...
                          """
                # same model calling and appending
                chunk_summary = query_llama(prompt, temperature=0.2)
                if chunk_summary:
                    final_summaries.append(chunk_summary)
                # Pause between chunk elaborations to avoid rate limiting
                if i < len(summary_chunks) - 1:  # No pause after last chunk
                    pause_time = 2 + random.uniform(0, 1)
                    print(f"Waiting {pause_time:.2f} seconds before next chunk...")
                    time.sleep(pause_time)
            #final summary obtained joining all partial mini-summaries
            final_summary = "\n".join(final_summaries)
        else:
            # If the size is manageable, let's make a single final summary
            print("Generating final summary...")
            prompt = f"""\
                      Combina questi riassunti parziali in un unico riassunto coerente e conciso:
                      '''{combined_summaries}'''

                      L'output deve avere il seguente formato:
                      Riassunto del Meeting:
                      - Punto 1...
                      - Punto 2...

                      Non scrivere nulla prima o dopo l'output.
                      """

            # Pause before final summary
            pause_time = 3 + random.uniform(0, 2)
            print(f"Waiting {pause_time:.2f} seconds before final summary generation...")
            time.sleep(pause_time)

            final_summary = query_llama(prompt, temperature=0.2)

        return final_summary
    else:
        # no partial_summaries
        return "Impossible to generate a summary."

def identify_tasks_from_chunks(chunks, max_chunk_size=6000):
    """
    Identify tasks from a list of chunks.

    Parameters:
    chunks(list): List of text chunks.
    max_chunk_size(int): Maximum size of each chunk.

    Returns:
    str: Generated tasks.
    """
    # Initialize a list to collect tasks from each chunk
    all_tasks = []

    for i, chunk in enumerate(chunks):
        print(f"Tasks extraction from chunk {i+1}/{len(chunks)}...")
        # prompt generation
        prompt = f"""
        Partendo da questa PARTE del meeting:
        '''{chunk}'''

        Identifica tutti i partecipanti presenti in questa parte e le attività assegnate ad essi.
        Tieni presente che questo è solo un frammento dell'intera discussione.
        Elenca solo le attività che trovi esplicitamente menzionate in questa parte.
        """

        # model calling, always low temperature to avoid too much creativity
        tasks = query_llama(prompt, temperature=0.2)
        if tasks:
            all_tasks.append(tasks) # appending tasks to the task list

        # Pause between chunk elaborations to avoid rate limiting
        if i < len(chunks) - 1:  # No pause after every chunk
            pause_time = 2 + random.uniform(0, 1)
            print(f"Waiting {pause_time:.2f} seconds before next chunk...")
            time.sleep(pause_time)

    # After processing all chunks, combine and deduplicate the tasks
    if all_tasks:
        combined_tasks = "\n\n".join(all_tasks)

        # If combined activities are still too long, we have to further divide them
        if len(combined_tasks) > max_chunk_size:
            print("Combined activities still too long, dividing again...")
            # Split back into manageable chunks
            tasks_chunks = split_text_into_chunks(combined_tasks, max_chunk_size)
            final_tasks_list = []

            # Process each split segment to deduplicate and format
            for i, tasks_chunk in enumerate(tasks_chunks):
                prompt = f"""
                Combina e deduplicizza questo elenco di attività estratte da un meeting:
                '''{tasks_chunk}'''

                L'output deve avere il seguente formato:
                Attività Identificate:
                - Partecipante 1: Attività (descrizione sintetica) in italiano...
                - Partecipante 2: Attività (descrizione sintetica) in italiano...

                Elimina duplicati e attività simili. Mantieni solo informazioni uniche e rilevanti.
                ASSICURATI CHE TUTTE LE ATTIVITÀ SIANO SCRITTE IN ITALIANO.
                """

                # model calling
                chunk_tasks = query_llama(prompt, temperature=0.2)
                if chunk_tasks:
                    final_tasks_list.append(chunk_tasks)

                # Pause between chunk elaborations to avoid rate limiting
                if i < len(tasks_chunks) - 1:
                    pause_time = 2 + random.uniform(0, 1)
                    print(f"Waiting {pause_time:.2f} seconds before next chunk...")
                    time.sleep(pause_time)

            final_tasks = "\n".join(final_tasks_list)
        else:
            # If the size is manageable, we do a single deduplication step
            print("Generating final tasks list...")
            prompt = f"""
            Combina e deduplicizza questo elenco di attività estratte da un meeting:
            '''{combined_tasks}'''

            L'output deve avere il seguente formato:
            Attività Identificate:
            - Partecipante 1: Attività (descrizione sintetica) in italiano...
            - Partecipante 2: Attività (descrizione sintetica) in italiano...

            Elimina duplicati e attività simili. Mantieni solo informazioni uniche e rilevanti.
            Se il partecipante non ha task assegnati, metti "Nessun task" come descrizione.
            ASSICURATI CHE TUTTE LE ATTIVITÀ SIANO DESCRITTE IN ITALIANO, non lasciare frasi in inglese.
            Non scrivere nulla prima o dopo l'output.
            """

            # Pause before final task list generation
            pause_time = 3 + random.uniform(0, 2)
            print(f"Waiting {pause_time:.2f} seconds before generating final tasks list...")
            time.sleep(pause_time)

            # final model calling
            final_tasks = query_llama(prompt, temperature=0.2)

        return final_tasks
    else:
        # If no tasks were identified in any chunk, return a fallback message
        return "No tasks detected."

# sometimes there are still language inconsistencies, so we define a translation funcion
def translate_text(text, target_language):
    """
    Translates text into the specified language using llama3 model.

    Parameters:
        text (str): Text to be translated.
        target_language (str): Target language code or name (e.g. 'it' or 'Italian').

    Returns:
        str: Translated text.
    """
    prompt = f"""Traduci completamente il seguente testo in {target_language}.
                È fondamentale che ogni frase e parola venga tradotta in {target_language}, anche i termini tecnici quando possibile.
                Se ci sono termini tecnici che è meglio mantenere in inglese, mantienili ma traduci tutto il resto della frase.

                Ecco il testo da tradurre:

                {text}

                Fornisci SOLO il testo tradotto in {target_language}, senza commenti aggiuntivi.
              """

    # model calling for translation
    translated_text = query_llama(prompt, temperature=0.2)

    return translated_text


def display_results(summary, tasks):
    """
    Format and display results in Markdown format.

    Parameters:
    summary(str): Generated summary.
    tasks(str): Generated tasks.

    Returns:
    str: Formatted Markdown output.
    """
    # format of preference
    markdown_output = f"""
    # Analisi della Trascrizione del Meeting

    ## Riassunto del Meeting
    {summary}

    ## Attività Identificate
    {tasks}
    """

    display(Markdown(markdown_output))

    return markdown_output

def save_results(output, filename="final_result.md"):
    """
    Save results to a file.

    Parameters:
    output(str): Formatted Markdown output.
    filename(str): Name of the file to save, default is "final_result.md".
    """
    with open(filename, "w", encoding="utf-8") as f:
        f.write(output)
    print(f"Results saved in '{filename}'")

def main(is_url = False):
    """
    Main function to execute the analysis.
    """
    # Maximum chunk (in characters) to avoid 413 Eroor
    MAX_CHUNK_SIZE = 3000

    if is_url:
      transcript_url = " https://raw.githubusercontent.com/Profession-AI/progetti-llm/refs/heads/main/Riassunto%20e%20identificazione%20delle%20attivit%C3%A0%20dalla%20trascrizione%20di%20un%20meeting/meeting_transcription.txt"

      print("Dowloading discussion file...")
      discussion = download_discussion(transcript_url)
    else:
      print("Reading local file...")

      # Reading local file content
      try:
          with open("meetings_folder/long_meeting.txt", "r", encoding="utf-8") as f:
              discussion = f.read()
      except Exception as e:
          print(f"Error in file reading: {e}")
          return


    if not discussion:
        print("Discussion file not found or empty")
        return

    print(f"Discussion loaded. Length: {len(discussion)} characters.")
    print("\nFirst few lines:")
    print(textwrap.fill(discussion[:200] + "...", width=100))

    # Dividing discussion in manageable chunks
    chunks = split_text_into_chunks(discussion, MAX_CHUNK_SIZE)

    print("\nSummary generation from chunks...")
    summary = generate_summary_from_chunks(chunks)

    print("\nExtracting tasks from chunks...")
    tasks = identify_tasks_from_chunks(chunks)

    # Original output
    print("\nAnalysis results:")
    output = display_results(summary, tasks)

    # Translation in italian, to avoid some strange multi-lingual outputs
    translated_output = translate_text(output, "italiano")
    output_to_save = translated_output

    # Saving results
    save_results(output=output_to_save, filename="meeting_analysis_result.md")

    print("\nAnalysis completed!")

# Execution
if __name__ == "__main__":
    main()

Reading local file...
Discussion loaded. Length: 24783 characters.

First few lines:
Maria Rossi: Buongiorno a tutti. La riunione di oggi sarà un po' più lunga del solito: dobbiamo
rivedere i risultati dei nuovi esperimenti, definire la versione alpha del modello da validare,
discuter...
Discussion divided in 9 chunks.

Summary generation from chunks...
Elaboration of chunk 1/9...
Waiting 2.74 seconds before next chunk...
Elaboration of chunk 2/9...
Waiting 2.94 seconds before next chunk...
Elaboration of chunk 3/9...
Waiting 2.34 seconds before next chunk...
Elaboration of chunk 4/9...
Waiting 2.67 seconds before next chunk...
Elaboration of chunk 5/9...
Waiting 2.40 seconds before next chunk...
Elaboration of chunk 6/9...
Waiting 2.01 seconds before next chunk...
Elaboration of chunk 7/9...
Waiting 2.02 seconds before next chunk...
Elaboration of chunk 8/9...
Waiting 2.61 seconds before next chunk...
Elaboration of chunk 9/9...
Combined Partial Summaries still too long, dividing agai


    # Analisi della Trascrizione del Meeting

    ## Riassunto del Meeting
    Ecco il riassunto coerente del meeting:

Riassunto del Meeting:

- Risultati esperimenti: Luca Bianchi ha presentato i risultati dei test della settimana, con varianti del modello LSTM, tra cui una con attenzione (attention layer) e una completamente trasformativa (Transformer). L'ibrido LSTM+attention si è comportato meglio, con RMSE 10.8 e varianza minore.

- Problemi di overfitting e anomalie: Il problema principale resta l'overfitting sui dati di dicembre e gennaio, probabilmente dovuto a dati esterni (vacanze natalizie, black friday, eventi imprevedibili). Elena Verdi ha esplorato le anomalie nei dati tra novembre e gennaio, trovando picchi molto marcati in alcuni negozi, probabilmente dovuti a promozioni locali non registrate nei metadati.

- Strategie per affrontare le anomalie: Sofia Gialli ha proposto di considerare una strategia a due stadi: prevedere se ci sarà un evento anomalo (binary classification), poi fare la regressione solo se la previsione è negativa. Giulio Ferri ha valutato che tecnicamente non ci sarebbero grandi complicazioni per implementare un sistema a due stadi.

- Impatti sull'utente finale: Chiara Neri ha sollevato la questione degli impatti sul comportamento dell'utente finale, che dipenderanno da come gestiremo il messaging.

- Intervallo di confidenza: Mostrare un intervallo di confidenza più largo in alcuni casi, addestrando una rete separata per stimare la varianza del prediction interval.

- Explainability: Utilizzare Temporal SHAP per calcolare la contribuzione feature per timestep, mostrando i fattori chiave per ogni previsione. Batch explainability per le previsioni giornaliere, una volta al giorno.

- Bias: Valutare il rischio di discriminazione territoriale, utilizzando tecniche di fairness-aware training come reweighting delle osservazioni o metodo di counterfactual fairness.

- Storage: Cancella le versioni di modelli con performance inferiori alla baseline, tenendo solo i top 5 per ogni architettura.

- Monitoraggio: Definire soglie chiare per il monitoraggio post-deployment, includendo latency, drift e outlier.

- Cena di lavoro: Scegliere un ristorante informale per la cena di lavoro, con opzioni gluten-free e parcheggio gratuito.

- Organizzazione della cena di squadra: Regali simbolici (tazze personalizzate con logo e nomi), punto d'incontro e modalità di arrivo.

- Partecipazione all'hackathon interno: Presentazione di un proof-of-concept di previsione in 24 ore.

- Definizione dei ruoli e delle responsabilità per l'hackathon: Marco Neri: ambiente Docker e predisposizione delle librerie. Elena Verdi: feature engineering e analisi esplorativa.

- Modelli e esperimenti: Luca Bianchi: esperimenti con Transformer e modelli LSTM. Sofia Gialli: esperimenti zero-shot per feature automatiche. Giulio Ferri: setup Knative e monitoraggio.

- Coordinamento e demo: Io e Chiara Neri: coordinamento, presentazione finale e demo.

- Dataset e augmentation: Luca Bianchi: utilizzo di dataset sintetici per aumentare la variabilità. Marco Neri: creazione di un Dockerfile per testare in locale. Maria Rossi: adattamento del modulo di augmentation per serie temporali. Sofia Gialli: contributo al modulo augment.py con metodologie di bootstrapping e rolling window mixup.

- Privacy e anonimizzazione: Giulio Ferri: anonimizzazione di pseudonimizzazione dei dati. Elena Verdi: notebook per mascherare nomi e indirizzi, da integrare nella pipeline.

- Fairness e report legale: Chiara Neri: call con il reparto legale su GDPR e fairness. Elena Verdi: analisi delle distribuzioni regionali e heatmap. Sofia Gialli: metriche fairness disaggregate.

- Altro: Marco Neri: problema di memory leak nel cluster GPU. Luca Bianchi: soluzione per chiudere la sessione manualmente in TensorFlow 2.10.

- Costi cloud e ottimizzazione: Giulio Ferri propone di spegnere i nodi spot quando non servono e usare spot invece di on-demand per i test non urgenti per evitare di superare il budget.

- Data drift detection: Sofia Gialli condivide la sua esperienza sul corso avanzato su MLOps di FaceAI e propone di utilizzare il test di Kolmogorov-Smirnov per identificare shift.

- Demo interattiva del modello: Chiara Neri comunica che il management vuole una demo interattiva del modello nel portale aziendale entro fine mese.

- Pianificato budget per il Q3.
Ecco il riassunto coerente del meeting:

Riassunto del Meeting:

- Budget e licenze: 11.000 € per formazione, 25.000 € per Weights & Biases, 4.000 € per Tableau, totale stimato 80.000 €, riserva per emergenze 20.000 €.
- Roadmap di release: lancio della versione beta previsto per il 15 luglio, integrazione entro fine giugno, rollout su staging il 30 giugno, test utente a partire dal 1-5 luglio.
- Copertura ferie estive: Chiara Neri in ferie dal 20 luglio al 3 agosto, Luca e Marco hanno il turno di copertura sulle emergenze.
- KPI e Obiettivi: Elena Verdi raggiunge il target di data quality, Marco Neri raggiunge il target di tempo di latenza end-to-end della pipeline, Sofia Gialli raggiunge il target di adozione di nuove tecniche di ricerca, Giulio Ferri raggiunge il target di implementazione del monitoraggio end-to-end, Chiara Neri raggiunge parzialmente il target di time-to-market delle feature.
- Azioni Correttive: incrementare il tasso di consolidamento degli esperimenti, ridurre l'impatto del GDPR sulla velocità dei report, analisi root-cause dei failure Airflow notturni, portare il prototipo GAN a validazione preliminare, raffinare la logica di alerting, ottimizzare il processo di QA.
- Benessere e Smart Working: discussione sulla gestione del lavoro da remoto e in ufficio, proposte per migliorare il lavoro di squadra e il work-life balance, introduzione di un "team wellbeing budget" per voucher per yoga o palestra e sessioni di mindfulness in ufficio.
- Lavoro ibrido e spazi di lavoro: proposta di lavoro da casa per la ricerca, modello ibrido 3/2 e spazio "silent room" per chi deve fare call o analisi senza distrazioni.
- Diversity & Inclusion: questione della rappresentanza di background differenziati, proposte di iniziative come stage per neolaureati di università diverse e regioni svantaggiate, seminari con speaker esterni e partecipazione a conferenze sulla diversity, azioni da intraprendere come definire programma stage e criteri di selezione, organizzare webinar con speaker esterni e identificare conferenze D&I per partecipazione 2025.

    ## Attività Identificate
    Ecco l'elenco delle attività deduplicate e tradotte in italiano:

Attività Identificate:
- Maria Rossi: Coordinare le attività e prendere decisioni
- Luca Bianchi: Presentare i risultati degli esperimenti e testare tecniche di fairness-aware training
- Elena Verdi: Presentare le scoperte sulle anomalie nei dati e lavorare su metriche robuste per classificare i giorni come “anomalie”
- Marco Neri: Verificare i log delle campagne digitali e predisporre un container Docker pre-configurato con tutte le librerie
- Sofia Gialli: Suggerire di utilizzare un modello a due stadi per prevedere eventi anomali e provare un approccio zero-shot con GPT per generare feature a partire da documentazione testuale
- Giulio Ferri: Valutare l'impatto tecnico di utilizzare un modello a due stadi sulla pipeline e implementare la funzione di auditing per loggare gli output “con zona modificata”
- Chiara Neri: Valutare il rischio di discriminazione territoriale e utilizzare gli insight per motivare le previsioni nelle dashboard interne

Nota: ho eliminato le attività relative alla cena e all'hackathon, poiché non sembrano essere attività lavorative rilevanti. Inoltre, ho tradotto le attività in italiano e ho eliminato i duplicati.
Ecco l'elenco delle attività deduplicate e tradotte in italiano:

Attività Identificate:
- Giulio Ferri: Preparare un breve runbook operativo per il team e suggerire di investire in GPU dedicate on-premise se i costi cloud diventano troppo alti.
- Sofia Gialli: Proof-of-concept data drift detection via KS test + autoencoder, proporre una licenza per la suite Weights & Biases e organizzare 3 webinar con speaker esterni entro luglio.
- Marco Neri: Predisporre un bucket separato per i dati di validazione continua, analizzare la root-cause dei failure Airflow notturni e considerare una licenza enterprise di Tableau.
- Luca Bianchi: Mettere in produzione un job schedulato con Airflow per calcolare KS ogni ora, endpoint REST + SHAP lightweight" per la demo, incrementare il tasso di consolidamento degli esperimenti e pianificare una revisione degli esperimenti in sospeso e decidere quali abbandonare o approfondire.
- Chiara Neri: Coordinamento, presentazione finale e demo, comunicazione sul management's request per la demo interattiva del modello, presentazione dei numeri per il budget, suggerire di mettere da parte 20 000 € come buffer e ottimizzare il processo di QA.
- Elena Verdi: Iscrivere due persone al corso "Advanced Time-Series Forecasting" e due persone al workshop "Explainable AI", eseguire 50 casi di test tra il 1° e il 5 luglio, ridurre l'impatto del GDPR sulla velocità dei report e definire programma stage e criteri di selezione.
- Maria Rossi: Riepilogare le spese, gestire la roadmap di release, discutere la revisione delle performance trimestrali del team e organizzare 3 webinar con speaker esterni entro luglio.

Nota: ho eliminato le attività duplicate e simili, e ho mantenuto solo le informazioni uniche e rilevanti.
    

Rate limiting (429). Retry in 4.43 seconds... (attempt 1/5)
Waiting 4.432859512639358 seconds before attempt 2...
Rate limiting (429). Retry in 9.67 seconds... (attempt 2/5)
Waiting 9.666150772663151 seconds before attempt 3...
Rate limiting (429). Retry in 20.19 seconds... (attempt 3/5)
Waiting 20.19037954750618 seconds before attempt 4...
Results saved in 'meeting_analysis_result.md'

Analysis completed!


# Final cleaning
this section is useful when we are working with big files, that will be divided in many chunks, this may cause some duplications and artifacts
what will be managed in this section:
- removing all the model artifacts (such as introductive or ending phrases)
- removing duplication in titles (derived from chunking division)

(This part can be removed, expanded or modified based on personal cases)

In [ ]:
def clean_meeting_results(file_path):
    """
    Clean the meeting results file by removing introductive phrases and titles.

    Parameters:
    file_path(str): File to clean path.

    Returns:
    str: cleaned file.
    """
    import re

    # Opens file
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except Exception as e:
        print(f"Error in reading file: {e}")
        return file_path

    print(f"Cleaning of file: {file_path}")

    # Removing introductive phrases like "Ecco il riassunto..."
    content = re.sub(r'Ecco (?:il|l\').*?:', '', content)

    # Removing repeated titles like "Riassunto del Meeting:" and "Attività Identificate:" that are not preceded by "#" (very risky, you have to trust the output of your model)
    content = re.sub(r'(?<!#)Riassunto del Meeting:', '', content)
    content = re.sub(r'(?<!#)Attività Identificate:', '', content)

    # Standardization ( - instead of *)
    content = content.replace('* ', '- ')

    # Rmoving multiple empty lines
    content = re.sub(r'\n{3,}', '\n\n', content)

    # Saving results in a new file, to avoid loss of information if something's wrong with the cleaning
    output_file = file_path.replace('.md', '_clean.md')
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"File cleaned and saved as: {output_file}")
    except Exception as e:
        print(f"Errore during file saving: {e}")
        return file_path

    return output_file





# Function calling
print("\nCleaning output file...")
clean_file = clean_meeting_results("meeting_analysis_result.md")


Cleaning output file...
Cleaning of file: meeting_analysis_result.md
File cleaned and saved as: meeting_analysis_result_clean.md


- reunite people with same name in task definition, in order to have a more clear and simple task list   
(Attention to homonyms!)

In [ ]:
def organize_tasks_by_person(file_path):
    """
    Read a markdown file and organize tasks by person.

    Parameters:
    file_path(str): Markdown file path.

    Returns:
    str: Re-organized file.
    """
    import re
    from collections import defaultdict

    # Opens file
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except Exception as e:
        print(f"Error during file reading: {e}")
        return file_path

    print(f"Reorganization of tasks in file: {file_path}")

    # Dividing content in sections (resume and task )
    sections = re.split(r'(## Riassunto del Meeting|## Attività Identificate)', content)

    # resume stays the same
    new_content = sections[0]  # Prima parte (titolo principale)

    for i in range(1, len(sections)):
        if "## Riassunto del Meeting" in sections[i]:
            new_content += sections[i] + sections[i+1]  # Adding header and summary content
        elif "## Attività Identificate" in sections[i]:
            # Task section processing
            tasks_section = sections[i]  # header
            if i+1 < len(sections):
                tasks_content = sections[i+1]  # task content
                # Task extraction
                tasks = re.findall(r'- (.*?)(?=\n- |\n\n|\Z)', tasks_content, re.DOTALL)

                # Dictinary to group tasks per person
                person_tasks = defaultdict(list)

                for task in tasks:
                    # Separing name from task description
                    match = re.match(r'(.*?):(.*)', task)
                    if match:
                        person = match.group(1).strip()
                        task_desc = match.group(2).strip()
                        person_tasks[person].append(task_desc)

                # Task section in new format
                reorganized_tasks = "\n\n"
                for person, tasks_list in person_tasks.items():
                    reorganized_tasks += f"{person}:\n"
                    for task in tasks_list:
                        reorganized_tasks += f"- {task}\n"
                    reorganized_tasks += "\n"

                # Adding new section
                new_content += tasks_section + reorganized_tasks

    # Removing multiple empty lines
    new_content = re.sub(r'\n{3,}', '\n\n', new_content)

    # Saving results in a new file, to avoid loss of information if something's wrong with the re-organization
    output_file = file_path.replace('.md', '_organized.md')
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(new_content)
        print(f"Reorganized file saved as: {output_file}")
    except Exception as e:
        print(f"Error during file saving: {e}")
        return file_path

    return output_file



# Function calling
print("\nTasks reorganization...")
organize_tasks_by_person(clean_file)



Tasks reorganization...
Reorganization of tasks in file: meeting_analysis_result_clean.md
Reorganized file saved as: meeting_analysis_result_clean_organized.md


'meeting_analysis_result_clean_organized.md'

# Conclusion
In this notebook we created a simple program that uses a free LLM to generate meeting summaries and tasks extraction for every people involved.

The solution is not perfect, it works well with this data, I suggest to try it with more different data and see if there are some critical issues

## Next Steps
- Using more powerful LLMs to see the difference in quality and in time response
- Compare LLM results with real summaries and tasks extraction results (ground truth), to see if this simple solution can substitute handwritten notes, or at least help
- Try to implement a RAG system with LangChain, trying to make questions and receive informations about past meeetings

